In [1]:
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

spark = sparknlp.start(gpu = True)

In [2]:
data = pd.read_csv("data/wikiner_dataset", sep='/n', header=None, index_col = False)
data.shape

/Users/todungnguyen/opt/anaconda3/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


(9381, 1)

In [3]:
phrases, ners = [], []
for d in data[0]:
    text = d.split()
    p, n = [], []
    for t in text:    
        word = t.split("|")
        p.append(word[0])
        n.append(word[2])
    phrases.append(' '.join(p))
    ners.append(n)

In [4]:
from pyspark.sql.types import StringType
df = spark.createDataFrame(phrases, StringType())
df.show()

+--------------------+
|               value|
+--------------------+
|Il assure à la su...|
|En 1905 , il occu...|
|Il a formé toute ...|
|Il devait diriger...|
|Il est notamment ...|
|En 1902 , il obti...|
|Son étudiant , Hr...|
|Ainsi il le dirig...|
|L' histoire de l'...|
|Après cette décou...|
|Carl Friedrich Ga...|
|En 1843 , William...|
|Georg Frobenius (...|
|Les espaces vecto...|
|Richard Brauer ( ...|
|Les algorithmes d...|
|L' algorithmique ...|
|Le savant arabe A...|
|L' utilisation du...|
|Ce découpage est ...|
+--------------------+
only showing top 20 rows



In [5]:
# https://towardsdatascience.com/training-a-contextual-spell-checker-for-italian-language-66dda528e4bf
assembler = DocumentAssembler()\
     .setInputCol("value")\
     .setOutputCol("document")

tokenizer = RecursiveTokenizer()\
     .setInputCols("document")\
     .setOutputCol("token")

## setLanguageModelClasses 
## dependent on the vocabulary size, and the model will use it to control the factoring in the language model
spellChecker = ContextSpellCheckerApproach()\
    .setInputCols("token")\
    .setOutputCol("corrected")\
    .setLanguageModelClasses(1650)\
    .setWordMaxDistance(3)\
    .setEpochs(10)
    
pipeline = Pipeline(
 stages = [
    assembler,
    tokenizer,
    spellChecker
 ])
model = pipeline.fit(df)

Py4JJavaError: An error occurred while calling o68.fit.
: java.lang.UnsatisfiedLinkError: no jnitensorflow in java.library.path
	at java.lang.ClassLoader.loadLibrary(ClassLoader.java:1867)
	at java.lang.Runtime.loadLibrary0(Runtime.java:870)
	at java.lang.System.loadLibrary(System.java:1122)
	at org.bytedeco.javacpp.Loader.loadLibrary(Loader.java:1683)
	at org.bytedeco.javacpp.Loader.load(Loader.java:1300)
	at org.bytedeco.javacpp.Loader.load(Loader.java:1123)
	at org.tensorflow.internal.c_api.global.tensorflow.<clinit>(tensorflow.java:12)
	at java.lang.Class.forName0(Native Method)
	at java.lang.Class.forName(Class.java:348)
	at org.bytedeco.javacpp.Loader.load(Loader.java:1190)
	at org.bytedeco.javacpp.Loader.load(Loader.java:1139)
	at org.tensorflow.NativeLibrary.load(NativeLibrary.java:64)
	at org.tensorflow.TensorFlow.init(TensorFlow.java:111)
	at org.tensorflow.TensorFlow.<clinit>(TensorFlow.java:126)
	at org.tensorflow.Graph.<clinit>(Graph.java:854)
	at com.johnsnowlabs.nlp.anno

In [ ]:
model.annotate("J'habitte à Allee Alfred Rocheray")